# Overview
* 以多项式曲线拟合(Polynomial Curve Fitting)为例，介绍了模式识别与机器学习里的一些概念。
* 概率论，决策论以及信息论的基础知识。

# Content
## 多项式曲线拟合的四个观点
如知乎某牛介绍，PRML这本书以四个不同观点贯穿整本书，即基本模型建模->极大似然估计->贝叶斯参数估计（inference，最大化参数后验概率）->贝叶斯预测（prediction，根据上一步得到的后验概率，把参数积分掉得到输出的概率分布）。
下面是多项式曲线拟合在上述四个观点下的解法：

### 基本模型建模
多项式函数写为：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.1.png)

多项式曲线拟合的问题可以定义为求参数**w**，使得下面error function最小：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.2.png)

由于误差函数是参数的二次函数，所以该一定能找到一个唯一的**w\***,使得误差函数最小。

剩下一个问题就是如何确定M，这是一种model comparison或model selection的概念。如果M选太小（例如M=1就是一条直线了），模型太简单，会造成under-fitting；如果M选太大，模型太复杂，会使得参数**w**的数值特别大，造成over-fitting。

模型太简单肯定不行，无论如何都拟合不了。那模型太复杂了怎么办？一个直观的想法是控制参数的数值大小，让它别太大了，于是自然地在误差函数中加入对参数值的惩罚，此时的误差函数为：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.4.png)

这种方法在机器学习中叫*regularization*，在统计学中叫*shrinkage*，在神经网络中叫*weight decay*。二次正则项的一个特殊情况叫做*ridge regression*.

### 极大似然估计
假设真实值与预测值的误差服从均值为0，precision为$\beta$的高斯分布，那么在给定输入x, 参数**w**的情况下，输出t的分布为：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.60.png)

如果我们的训练数据包含N个样本点，且每个样本点都是从分布(1.60)中独立产生的，那么该训练数据的对数似然函数为：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.62.png)

通过求导，我们可以得到$w_{ML}$以及$\beta_{ML}$。可以发现，**在noise服从高斯分布的假设下，极大似然估计结果等价于以最小二乘为误差函数的基本模型建模结果。**

此时，给定一个新的输入x，输出t的概率(predictive distribution)可以写为如下式子： ![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.64.png)

### 贝叶斯参数估计
在极大似然估计中，我们对参数**w**没有任何先验假设，贝叶斯估计就是对要学习的参数作了先验假设。我们假设参数**w**的每一维相互独立（协方差矩阵只有对角不为0），即服从：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.65.png)

根据贝叶斯理论，参数**w**的后验分布正比于先验分布x似然函数，即：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.66.png)

我们要找到给定训练样本下参数**w**可能性最大的值，即最大化后验概率（maximum posterior or MAP）。经过求解，该最大后验概率值可通过下列式子得到：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.67.png)

可以发现，**在贝叶斯参数估计的观点下，会自动产生一个正则化项**

### 贝叶斯预测
到贝叶斯参数估计为止，我们还只是得到参数的点估计值。其实，每一种参数值都有一定概率出现，而一个输入在每一种参数值下都会得到不同的输出分布。因此，我们需要把所有参数的可能取值积分掉，得到prediction distribution的终极形式：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.68.png)

这个式子有解析解为：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.69.png)

**可以发现，预测值t的不确定性（方差）由两部分决定，一部分和极大似然估计一样由$\beta_{ML}^{-1}$决定，另一部分由参数**w**的不确定性决定，这一部分是贝叶斯预测独有的。**

## 模型选择
如之前所述，在多项式曲线拟合的例子中，多项式的秩M决定模型的自由参数个数，正则化系数控制模型的有效复杂度（可以控制参数的值）。在实际应用中，我们如何选择这些决定模型复杂度的参数呢？这就是模型选择问题。通常有几种方式：
1. 如果数据量足够大，可以将数据分为训练集和验证集。在训练集上训练不同的模型（或者相同模型不同取值的参数），在验证集上选择表现最好的模型。
2. 通常数据量不会很大，这时可以用交叉验证。如果验证集的样本个数为1，则称为leave-one-out。

## 维度灾难
当输入变量维度很高时，参数组合的个数幂数增长。其实这节没怎么看懂...

## 决策论  
假设有个输入向量**x**以及输出向量**t**，如果我们已经从训练样本中计算出它们的联合分布$p(x, t)$（inference阶段），那么我们应该如何选择**t**的值呢？这就是决策论要解决的问题。

### 分类问题
对于分类问题，我们需要一个rule将输入**x**分到不同的类别中。这种rule会将输入空间划分成$R_k$，称为decision region，这些regions的边界称为decision boundaries或decision surfaces.

假设对于一个新的输入**x**，真实分类是$C_k$，我们将其分类到$C_j$获得的惩罚是$L_{kj}$。例如，将癌症病人诊断为健康人获得的惩罚是1000，而将健康人诊断为癌症病人的惩罚是10。因此，我们希望最小化loss function: ![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.80.png)

这个loss function就是整个输入空间的期望损失。因为每个输入**x**会被独立地分到一个decision region $R_j$，所以我们的目标就是寻找一个类别j，使得$\sum_{k}{L_{kj}p(x, C_{k})}$最小。由于$p(x,C_{k})=p(C_{k}|x)p(x)$且对于每个**x**，$p(x)$相同，因此目标就转化为：对于每一个输入**x**，我们应该将其归类为使得下列式子最小的$C_j$。
$$\sum_k{L_{kj}}p(C_k|x)$$

参考习题1.24可以发现，当$L_{kj}=1-I_{kj}$时，最小化该loss function等价于最小化分类错误率。

#### Reject Option
可以看出，后验概率$p(C_k|x)$的最大值决定了分类错误率，最大值越小，误分类率越高。因此，为了控制误分类率，我们需要控制后验概率的最大值。当后验概率的最大值小于或等于$\theta$时，我们要拒绝make decision，这称为reject option。很容易发现，当$\theta=1$时，我们reject所有input；当$\theta<1/k$时，我们接受所有input。

#### inference and decision
至此，我们将分类问题分为两步：从训练数据中求类别的后验概率$p(C_k|x)（$inference stage）和根据后验概率决定输出类别（decision stage）。当然，我们也可以直接学习一个从输入到输出类别的函数，该函数称为discriminant function。

因此，在实际应用中，有三种不同方法求解分类问题，这三种方法从复杂到容易：
1. 先求输入和输出的联合分布并用贝叶斯公式求得输出的后验概率p(C_k|x)，再用决策论求解最优类别。这种求解联合分布的模型称为生成模型（因为有了联合分布，我们可以生成出数据点）
2. 直接求得类别得后验概率，再用决策论求解最优类别。这种直接求解后验概率得模型称为判别模型。
3. 直接求解discriminant function。

三种方法对比：
第一种方法由于可以得到联合分布，积分后可以得到输入变量x的概率，因此可以作异常点检测。
前两种方法都可以得到后验概率，好处是：
1. 决策的损失函数可以随时调整。
2. 可以有reject option。
3. 可以在合成的数据集上求联合分布，再apply到真实训练集上，只要乘上类别的先验，就可以得到后验概率。
4. 不同输入的模型可以直接结合。

### 回归问题
对于回归问题，决策过程是对于每一个输入x，找到输出值y(x)，使得期望损失最小，即：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.86.png)

如果用最小二乘作为损失函数，那么期望损失可以写为：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.87.png)

根据习题1.25，用变分法对该损失函数关于y(x)求导，得：![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.89.png)

当然，也可以选择其他函数作为损失函数，例如Minkowski loss: ![](https://raw.githubusercontent.com/wzpfish/PRML-notes/master/graphs/Chapter01/1.91.png)

因此，与分类问题一样，有三种方法可以求解回归问题：
1. 先求联合分布$p(x, t)$并用贝叶斯公式求得输出的后验概率$p(t|x)$，然后通过最小化损失函数求解$y(x)$.
2. 直接求输出的后验概率，然后通过最小化损失函数求解$y(x)$.
3. 直接求解regression function $y(x)$.

例如在上面的多项式曲线拟合例子中，我们最后得到了后验概率$p(t|x)$，可以用步骤2来求最后的输出$t$。

PS: 最后，信息论和概率论的知识就不记了，都是些基础，忘了直接翻书就好。